In [1]:
import numpy as np
import pandas as pd
from GeoSolver import GeoSolver

In [ ]:
class Object():
    """Описывает один материал в заказе"""
    
    def __init__(self, material, count):
        """
        материал - колличество
        """
        self.material = material
        self.count = count

In [ ]:
class Lot():
    
    """Описывает один лот"""
    
    def __init__(self, material, count, date, coords, providers):
        """
        материал и его кол-во, срок поставки, клиента (грузополучателя)
        Создает объет матерала
        Формирует словарь возможных поставщиков и кол-во товаров, которые они могут 
        закупить из переданной таблцы материал-поставщик (мб None!)
        Инициализирует географический центр
        Инициализируется минимальная и максимальная дата доставки
        """
        
        self.__content = [Object(material, count)]
        self.__min_date, self.__max_date = date, date
        self.__center = coords
        self.__providers = {p : 1 for p in providers}
        self.__order_number = 1 # Количество позиций
    
    def get_distance(self, other):
        """
        Если None, то None
        Принимает на вход еще один лот, возвращает расстояние между ними
        Смотрит на разницу минимальной и максимальной даты доставки  <= 30
        Проверяется Условие Качества (50% выкупают 50% если объединить)
        Считается расстояние
        Возвращается расстояние 
        """
        pass
    
    def merge(self, other):
        """
        Сливает other в теекущий лот
        Обновляет центр, минимальные даты
        Сливает словари
        Добавляет список товаров
        """
        pass
    
    def providers_merge(self):
        """
        получает словарь поставщиков и сливает его со своим
        """
        pass
    
    def get_center(self):
        pass
    
    def get_dates(self):
        pass

In [ ]:
class Solver():
    """
    Модель 1. Агломеративная кластеризация
    """
    
    def __init__(self):
        """
        Подгружает таблицу грузополучатель/координаты
        Подгружает справочник материал/поставщик
        Инифицализирует пустым списком список лотов
        """
        self.__coords = pd.read_csv("./Data/coords_test.csv")
        self.__providers = pd.read_csv("./Data/Кабель-справочник-МТР-refactored.csv")
        self.__lots = []
        
    
    def get_lots(self, filename):
        """
        Получает имя файла в формате csv
        Запускает __file_handler()
        Запускает __construct_lots()
        возвращает то, что лежит в лотах
        """
        
        self.__file_handler(filename)
        self.__construct_lots()
        
        return self.__lots
    
    def __file_handler(self, filename):
        """
        Открывает csv файл, делает датафрейм
        Проходит последовательно по строкам датафрейма, создавая лоты
        Лоты создаются в список self.__lots
        """
        
        data = pd.read_csv(filename)
        for i in df.index:
            material, count, client, date = data['Материал'][i], data['Общее количество'][i], \
                                            data['Клиент'][i], data['Срок поставки'][i]
            coords = self.__coords[self.__coords['Код грузополучателя'] == client]['Широта', 'Долгота']
            providers = self.__providers[self.__providers['Материал'] == material]['Поставщики']
            self.__lots.append(Lot(material, count, date, coords, providers))
        
        data.close()
    
    def __construct_lots(self):
        """
        Идет цикл с проверкой удовлетворения условия
        {
        вызввается __calc_min_distance(self)
        если None то мы закончили
        иначе lot[i].merge(lot[j])
        }
        """
        
        best_option = self.__calc_min_distance()
        while best_option is not None:
            i, j = best_option[0], best_option[1]
            self.__lots[i].merge(self.__lots[j])
            self.__lots[j] = None
            best_option = self.__calc_min_distance()
            

    def __calc_min_distance(self):
        """
        Создает булевый список посещенных лотов
        Проходится по списку лотов
        Между каждой парой вызывает lot.get_distance(other)
        Запоминает и возвращает оптимальную пару или None
        """
        min_dist, opt1, opt2 = 1e10, -1, -1
        for i, first in enumerate(self.__lots):
            for j, second in enumerate(self.__lots[i+1:]):
                dist = first.get_distance(second)
                if dist < min_dist:
                    min_dist = dist
                    opt1, opt2 = i, j
                    
        if opt1 == -1:
            return None
        return (i, j)